In [6]:
pip install face_recognition

Note: you may need to restart the kernel to use updated packages.


In [5]:
import datetime

# ดึงเวลาในปัจจุบัน
current_time = datetime.datetime.now().time()  # เวลาในรูปแบบ HH:MM:SS

In [8]:
!pip install opencv-python

  Using cached opencv_python-4.10.0.84-cp37-abi3-win_amd64.whl.metadata (20 kB)
Using cached opencv_python-4.10.0.84-cp37-abi3-win_amd64.whl (38.8 MB)


In [13]:
pip install deepface

Note: you may need to restart the kernel to use updated packages.


In [17]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [20]:
pip install tf-keras


   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
    --------------------------------------- 0.0/1.7 MB 495.5 kB/s eta 0:00:04
   ----- ---------------------------------- 0.2/1.7 MB 2.0 MB/s eta 0:00:01
   --------------- ------------------------ 0.7/1.7 MB 4.3 MB/s eta 0:00:01
   --------------------------- ------------ 1.2/1.7 MB 6.3 MB/s eta 0:00:01
   ------------------------------------- -- 1.6/1.7 MB 6.5 MB/s eta 0:00:01
   ---------------------------------------- 1.7/1.7 MB 6.4 MB/s eta 0:00:00


In [3]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.18.0


In [23]:
pip install mysql-connector-python

   ---------------------------------------- 0.0/16.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/16.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/16.1 MB 435.7 kB/s eta 0:00:37
   ---------------------------------------- 0.1/16.1 MB 770.8 kB/s eta 0:00:21
    --------------------------------------- 0.2/16.1 MB 1.5 MB/s eta 0:00:11
   - -------------------------------------- 0.5/16.1 MB 3.0 MB/s eta 0:00:06
   -- ------------------------------------- 1.1/16.1 MB 4.8 MB/s eta 0:00:04
   ---- ----------------------------------- 1.6/16.1 MB 6.2 MB/s eta 0:00:03
   ----- ---------------------------------- 2.0/16.1 MB 6.2 MB/s eta 0:00:03
   ----- ---------------------------------- 2.4/16.1 MB 6.6 MB/s eta 0:00:03
   ------- -------------------------------- 2.9/16.1 MB 6.9 MB/s eta 0:00:02
   -------- ------------------------------- 3.3/16.1 MB 7.1 MB/s eta 0:00:02
   --------- ------------------------------ 4.0/16.1 MB 7.7 MB/s eta 0:00:02
   ------

In [ ]:
import cv2
import os
import datetime
import time
from deepface import DeepFace
import mysql.connector
import datetime

# ดึงเวลาในปัจจุบัน
current_time = datetime.datetime.now().time()  # เวลาในรูปแบบ HH:MM:SS

# กำหนดโฟลเดอร์สำหรับบันทึกภาพที่แค็ปมาจากกล้อง
capture_folder = r"C:\xampp\htdocs\log_studen_van\capture_img"
if not os.path.exists(capture_folder):
    os.makedirs(capture_folder)

# กำหนดโฟลเดอร์สำหรับเก็บภาพนักเรียนที่ใช้ในการเปรียบเทียบ
image_folder_path = r"C:\xampp\htdocs\log_studen_van\img"

# กำหนดโซนการตรวจจับใบหน้า
zone_x, zone_y, zone_w, zone_h = 100, 100, 400, 300

# โหลด Haar Cascade สำหรับตรวจจับใบหน้า
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# เปิดกล้อง
cap = cv2.VideoCapture(0)

# กำหนดเวลาคูลดาวน์ (5 วินาที)
cooldown_time = 5
last_capture_time = 0

# ฟังก์ชันเชื่อมต่อฐานข้อมูล
def connect_database():
    return mysql.connector.connect(
        host="localhost",
        user="root",
        password="",
        database="van_project"
    )

# ฟังก์ชันบันทึกเวลาเข้า
def log_entry(student_id):
    conn = connect_database()
    cursor = conn.cursor()
    cursor.execute("""
        INSERT INTO student_attendance (student_id, entry_time)
        VALUES (%s, %s)
    """, (student_id, datetime.datetime.now()))
    conn.commit()
    cursor.close()
    conn.close()
    print(f"บันทึกเวลาเข้าให้ Student ID: {student_id}")

# ฟังก์ชันบันทึกเวลาออก
def log_exit(student_id):
    conn = connect_database()
    cursor = conn.cursor()
    cursor.execute("""
        UPDATE student_attendance 
        SET exit_time = %s 
        WHERE student_id = %s AND exit_time IS NULL
    """, (datetime.datetime.now(), student_id))
    conn.commit()
    cursor.close()
    conn.close()
    print(f"บันทึกเวลาออกให้ Student ID: {student_id}")

# ฟังก์ชันบันทึกเวลาเข้า-ออกตามช่วงเวลา
def log_attendance(student_id):
    current_time = datetime.datetime.now().time()  # เวลาปัจจุบันในรูปแบบ HH:MM:SS

    # กำหนดช่วงเวลาต่างๆ
    morning_entry_start = datetime.time(6, 0)
    morning_exit_start = datetime.time(8, 0)
    afternoon_entry_start = datetime.time(15, 0)
    afternoon_exit_start = datetime.time(17, 0)

    # ตรวจสอบช่วงเวลาและบันทึกเวลาเข้า-ออก
    if morning_entry_start <= current_time < morning_exit_start:
        log_entry(student_id)
    elif morning_exit_start <= current_time < afternoon_entry_start:
        log_exit(student_id)
    elif afternoon_entry_start <= current_time < afternoon_exit_start:
        log_entry(student_id)
    elif current_time >= afternoon_exit_start:
        log_exit(student_id)

# ฟังก์ชันเปรียบเทียบภาพโดยใช้ DeepFace
def compare_images(captured_face_path, database_image_path):
    try:
        result = DeepFace.verify(img1_path=captured_face_path, img2_path=database_image_path, model_name="Facenet")
        similarity = result['verified']  # True หมายถึงจับคู่สำเร็จ, False หมายถึงไม่สำเร็จ
        distance = result['distance']    # ระยะห่างระหว่าง face embeddings ยิ่งน้อยยิ่งคล้าย
        return similarity, distance
    except Exception as e:
        print(f"Error comparing images: {e}")
        return False, None

# ตรวจสอบการเชื่อมต่อฐานข้อมูล
try:
    conn = connect_database()
    if conn.is_connected():
        print("เชื่อมต่อฐานข้อมูลสำเร็จ")
    conn.close()
except Exception as e:
    print(f"การเชื่อมต่อฐานข้อมูลล้มเหลว: {e}")

try:
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        current_time = time.time()

        # วาดกรอบโซนการตรวจจับ
        cv2.rectangle(frame, (zone_x, zone_y), (zone_x + zone_w, zone_y + zone_h), (0, 255, 0), 2)

        # แปลงเป็นสี Grayscale เพื่อให้ตรวจจับใบหน้าได้ง่ายขึ้น
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # ตรวจจับใบหน้า
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

        for (x, y, w, h) in faces:
            # ตรวจสอบว่าใบหน้าอยู่ภายในโซนการตรวจจับหรือไม่
            if x > zone_x and (x + w) < (zone_x + zone_w) and y > zone_y and (y + h) < (zone_y + zone_h):
                # แสดงกรอบรอบใบหน้าในภาพหลัก
                cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

                # ตรวจสอบว่าผ่านคูลดาวน์หรือไม่เพื่อแคปภาพ
                if current_time - last_capture_time >= cooldown_time:
                    # แคปใบหน้าในกรอบที่ตรวจพบ
                    face_img = frame[y:y+h, x:x+w]
                    # บันทึกภาพที่แคปไว้เป็นไฟล์ชั่วคราวเพื่อใช้เปรียบเทียบ
                    captured_face_path = os.path.join(capture_folder, "captured_temp.jpg")
                    cv2.imwrite(captured_face_path, face_img)

                    # เปรียบเทียบใบหน้าที่แคปได้กับใบหน้าในฐานข้อมูล
                    best_match_id = None
                    lowest_distance = float("inf")
                    
                    for db_filename in os.listdir(image_folder_path):
                        db_image_path = os.path.join(image_folder_path, db_filename)
                        is_match, distance = compare_images(captured_face_path, db_image_path)
                        print(f"กำลังเปรียบเทียบกับ {db_filename}, distance: {distance}")

                        # ตรวจสอบว่าภาพนี้มีระยะห่างต่ำที่สุดและจับคู่สำเร็จหรือไม่
                        if is_match and distance < lowest_distance:
                            lowest_distance = distance
                            best_match_id = os.path.splitext(db_filename)[0]
                            print(f"อัปเดต best_match_id เป็น {best_match_id} ด้วยระยะห่าง {lowest_distance:.4f}")

                    # กำหนดเกณฑ์ระยะห่างขั้นต่ำ (ค่า 0.6 สามารถปรับได้ตามความเหมาะสม)
                    if best_match_id is not None and lowest_distance < 0.6:
                        print(f"จับคู่กับ Student ID: {best_match_id} ด้วยระยะห่าง {lowest_distance:.4f}")
                        log_attendance(best_match_id)  # บันทึกเวลาเข้า-ออกตามช่วงเวลา
                    else:
                        print("ไม่พบการจับคู่ใบหน้าในฐานข้อมูล")

                    # อัปเดตเวลาการตรวจจับครั้งล่าสุด
                    last_capture_time = current_time

        # แสดงผลจากกล้อง
        cv2.imshow('Face Detection and Capture', frame)

        # กด 'q' เพื่อออกจากลูป
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

except KeyboardInterrupt:
    print("Stopped by User")

# ปิดกล้อง
cap.release()
cv2.destroyAllWindows()
